In [1]:
!pip install datasets transformers faiss-cpu tqdm langchain langchain-community openai langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [1]:
import os
import json
import pickle
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
import os
import faiss
from google.colab import files  # Colab에서만 필요, 로컬에서는 제거

#### Dataset Load

In [2]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "XXXXXXXX"

# 데이터셋 로드
!pip install --upgrade datasets huggingface_hub fsspec

In [3]:
from datasets import load_dataset
dataset = load_dataset("squad_kor_v1")
print(f"데이터셋 로드 완료: {len(dataset['train'])} 학습 샘플, {len(dataset['validation'])} 검증 샘플")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.29k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

데이터셋 로드 완료: 60407 학습 샘플, 5774 검증 샘플


#### Document 전처리

In [4]:
# 문서 준비
documents = []

# 학습 데이터와 검증 데이터 모두 활용
for split in ['train', 'validation']:
    for item in tqdm(dataset[split]):
        doc = {
            'id': item['id'],
            'title': item['title'],
            'context': item['context'],
            'question': item['question'],
            'answers': item['answers']
        }
        documents.append(doc)

# 중복 제거 (context 기준)
unique_contexts = {}
unique_docs = []

for doc in documents:
    if doc['context'] not in unique_contexts:
        unique_contexts[doc['context']] = doc
        unique_docs.append(doc)

print(f"전처리된 문서 수: {len(documents)} (중복 제거 후: {len(unique_docs)})")

100%|██████████| 5774/5774 [00:00<00:00, 13975.54it/s]

전처리된 문서 수: 66181 (중복 제거 후: 10563)


#### 임베딩 모델 로딩
- 임베딩 모델 선택 벤치마크 참고
- https://huggingface.co/dragonkue/snowflake-arctic-embed-l-v2.0-ko

아래의 두 임베딩 모델은 GPU 램 이슈로 실행 실패.. 코랩 결제 불가 이슈로 인한
- dragonkue/snowflake-arctic-embed-l-v2.0-ko
- dragonkue/BGE-m3-ko

아래의 모델은 성공
- nlpai-lab/KURE-v (서버에서 돌리는데 이슈가 있음 - gpu 설정 이슈)

다만, 사용자 쿼리 임베딩 차원 불일치 문제 및 gpu 이슈로 "all-MiniLM-L6-v2" 모델로 변경

성능이 너무 안좋아서 openai embedding model 사용...
- text-embedding-ada-002

In [8]:
# 중복 제거 후의 문서 목록 사용
documents = unique_docs

# OpenAI 임베딩 클라이언트 초기화
embed_model = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)
print(f"OpenAI 임베딩 모델 초기화 완료 (text-embedding-ada-002)")

# 문서를 LangChain Document 객체로 변환
docs = [
    Document(page_content=d['context'], metadata={'id': d['id'], 'title': d['title']})
    for d in documents
]

print(f"문서 변환 완료: {len(docs)} 문서")

OpenAI 임베딩 모델 초기화 완료 (text-embedding-3-small)
문서 변환 완료: 10563 문서


#### 임베딩 생성 및 FAISS 인덱스 구축
- 고차원 벡터 공간에서 빠른 유사도 검색을 위해 특별히 설계
- 임베딩과 같은 고차원 벡터(예: 768차원, 1024차원)에서 최근접 이웃(Nearest Neighbors)을 찾는 작업에 최적화

* 이러한 이유때문에 FAISS 사용

In [9]:
# 배치 처리로 임베딩 생성
print("임베딩 생성 중...")
batch_size = 20  # OpenAI API 호출 최적화를 위한 배치 크기
all_embeddings = []
all_docs = []

for i in tqdm(range(0, len(docs), batch_size)):
    batch_docs = docs[i:i + batch_size]
    all_docs.extend(batch_docs)

    # 임베딩 생성 (OpenAI API 호출)
    texts = [doc.page_content for doc in batch_docs]
    embeddings = embed_model.embed_documents(texts)
    all_embeddings.extend(embeddings)

# FAISS 벡터 저장소 생성
print("FAISS 벡터 저장소 생성 중...")
dimension = len(all_embeddings[0])  # OpenAI의 차원 (1536)
print(f"임베딩 차원: {dimension}")

임베딩 생성 중...


100%|██████████| 529/529 [09:45<00:00,  1.11s/it]

FAISS 벡터 저장소 생성 중...
임베딩 차원: 1536


#### 데이터 저장

In [10]:
# 각 문서와 임베딩을 튜플로 저장
text_embeddings = [
    (doc.page_content, all_embeddings[i])
    for i, doc in enumerate(all_docs)
]

# 메타데이터 준비
metadatas = [doc.metadata for doc in all_docs]
ids = [str(doc.metadata['id']) for doc in all_docs]

# FAISS 벡터 저장소 생성
vectorstore = FAISS.from_embeddings(
    text_embeddings,
    embedding=embed_model,
    metadatas=metadatas,
    ids=ids
)

# 저장
os.makedirs('korquad_data_openai', exist_ok=True)
vectorstore.save_local(
    folder_path='korquad_data_openai',
    index_name='document_index'
)

print("✅ OpenAI 임베딩 기반 LangChain-FAISS 인덱스 및 메타데이터 생성 완료")
print("   - korquad_data_openai/document_index.faiss")
print("   - korquad_data_openai/document_index.pkl")

# 코랩에서 다운로드 (필요한 경우)
files.download('korquad_data_openai/document_index.faiss')
files.download('korquad_data_openai/document_index.pkl')

✅ OpenAI 임베딩 기반 LangChain-FAISS 인덱스 및 메타데이터 생성 완료
   - korquad_data_openai/document_index.faiss
   - korquad_data_openai/document_index.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### 다운로드

In [ ]:
!zip -r korquad_data.zip korquad_data/

files.download('korquad_data.zip')

  adding: korquad_data/ (stored 0%)
  adding: korquad_data/document_index.pkl (deflated 64%)
  adding: korquad_data/document_index.faiss (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>